# Mangrove Coverage: 1988-2025
The code in this file creates an HTML file that visualizes the mangrove forest coverage from 1988 to 2025 on an interactive Earth Engine map. It adds individual layers for each year of the mangrove extent, allowing the user to view and compare areas of mangrove loss. 

In [40]:
# --- Run underlying python scripts
import GEN01_GEE_Authenticate
import GEN02_AOI as  aoi_def
import GEN03_helper_functions as HF
import GEN04_mangrove_layers as ML 
import GEN05_HTML_layout as HTML 
import GEN06_shapefiles as shp


In [ ]:
import ee, os

# --------- Activate this before pushing to GitHub --------------------
SERVICE_ACCOUNT = os.environ.get("GEE_SERVICE_ACCOUNT")
KEY_FILE = "key.json"   # GitHub Actions writes the secret here

credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
ee.Initialize(credentials)

# --------- Activate when using in VS Code --------------------
# ee.Authenticate()
# ee.Initialize(project="gee-mekong-map")

In [ ]:
# ------------- Packages & GEE ------------
import ee                                       # Google Earth Engine API for Python
import geemap.foliumap as geemap_folium         # Geemap module integrating Folium maps
import folium                                   # Interactive mapping library based on Leaflet.js
from branca.element import Element              # Low-level HTML/JS elements for Folium/Branca maps
import geopandas as gpd                         # Spatial data handling (GeoDataFrames)

# small bug fix on GEE initialization
geemap_folium.ee_initialize = lambda *args, **kwargs: None


In [43]:
# Create a folium-based map
Map = geemap_folium.Map(center=[9.2, 105.75], zoom=11, initialize=False)

# Adds the base satellite image layer on the map
Map.add_basemap('SATELLITE')

In [44]:
# --------------- Dictionary of mangrove coverage masks (updated years) ---------------
mangrove_coverage = {
    "Mangrove coverage 1988": ML.mangrove_1988,
    "Mangrove coverage 1992": ML.mangrove_1992,
    "Mangrove coverage 1997": ML.mangrove_1997,
    "Mangrove coverage 2001": ML.mangrove_2001,
    "Mangrove coverage 2005": ML.mangrove_2005,
    "Mangrove coverage 2010": ML.mangrove_2010,
    "Mangrove coverage 2015": ML.mangrove_2015,
    "Mangrove coverage 2020": ML.mangrove_2020,
    "Mangrove coverage 2025": ML.mangrove_2025,
}

for label, img in mangrove_coverage.items():

    # Add to interactive map
    Map.addLayer(
        img.updateMask(img),
        {"palette": [ML.color_mangrove_coverage]},
        label
    )


Add legend, scalebar, north arrow, Living Lab logo

In [45]:
# ------- Add legend      --------
legend_title = "Mangrove Coverage Map (1988-2025)"

HTML.add_folium_legend(Map, legend_title, HTML.legend_dict_mangrove_COVERAGE, style=HTML.style)

# ------- Add scalebar     -------
Map.add_child(HTML.ScaleBar(font_size="14px"))

# ------- Add north arrow --------
HTML.add_north_arrow(Map, position="bottomleft", arrow_size="35px", text_size="25px")

# ------- Add Living LAb logo -----
Map.get_root().html.add_child(folium.Element(HTML.logo_html))

Add shapefiles

In [46]:
# Add the commune boundaries to the map
folium.GeoJson(
    shp.geojson_commune,
    name="Commune boundaries",
    style_function=shp.commune_style,
    ).add_to(Map)

#  Add the sea dikes group to the map
shp.sea_dikes_group.add_to(Map)

# Add the breakwater group to the map
shp.breakwaters_group.add_to(Map)

# other human activities
shp.Extra_group.add_to(Map)
folium.LayerControl().add_to(Map)   

In [47]:
Map.to_html("Mangrove_COVERAGE_map.html")
